## パッケージのインストールと読み込み


In [1]:
#| label: このファイルで利用する不足パッケージのインストール
#| eval: false
#| echo: true
install.packages(c("gghighlight", 
                   "ggrepel", "palmerpenguins", "patchwork",
                   "zipangu"))
install.packages("ssdse", 
                 repos = c("https://uribo.r-universe.dev", "https://cloud.r-project.org"))

In [2]:
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(patchwork) # 複数のグラフの配置調整
library(gghighlight) # グラフの一部をハイライト
library(ggrepel) # ラベル表示 (geom_text_repel, geom_label_repel)
library(palmerpenguins)
library(zipangu)
data(penguins)

In [3]:
# データフレームから四国4県のデータを抽出しやすくする関数を定義する
filter_shikoku <- function(data, var = prefecture) {
  data |>
    dplyr::filter({{ var }} %in% c("Tokushima-ken", "Kagawa-ken", "Ehime-ken", "Kochi-ken")) |> 
    dplyr::mutate({{ var }} := forcats::fct_relevel({{ var }}, "Tokushima-ken", "Kagawa-ken", "Ehime-ken", "Kochi-ken"))
}

## データ可視化の重要性

### アンスコムの例


In [4]:
anscombe

anscombe_long <-
  anscombe |>
  tidyr::pivot_longer(
    tidyselect::everything(),
    names_to = c(".value", "set"),
    names_pattern = "(.)(.)")

anscombe_long |>
  group_by(set) |>
  group_map(
    ~ ggplot(.x, aes(x, y)) +
      geom_point() +
      geom_smooth(method = lm, se = FALSE)) |>
  wrap_plots(ncol = 4)

## ggplot2によるグラフ作成

::: {.callout-warning}

情報センターJupyterHubではRの作図中に日本語を利用すると文字化けします。対策を検討中...。

:::


In [5]:
#| label: step1 データの指定
#| eval: false
#| echo: true
# ペンギンデータを利用
ggplot(data = penguins)

In [6]:
#| label: step2　審美的要素とデータの紐付け
#| eval: false
#| echo: true
ggplot(data = penguins) +
  # x軸、y軸に対応するデータを指定
  aes(x = flipper_length_mm, 
        y = bill_length_mm)

In [7]:
#| label: step3 ジオメトリレイヤを選択
#| eval: false
#| echo: true
ggplot(data = penguins) +
  aes(x = flipper_length_mm, 
        y = bill_length_mm) +
  # 散布図として描画
  geom_point()

In [8]:
#| label: step4 審美的要素とデータの紐付け
#| eval: false
#| echo: true
ggplot(data = penguins) +
  aes(x = flipper_length_mm, 
        y = bill_length_mm,
        # speciesでの散布図の塗り分け
        color = species) +
  geom_point()

## 多様なグラフ表現

独立行政法人統計センターによる様々な分野の都道府県別・時系列データを集めたデータセット（[SSDSE-県別推移](https://www.nstac.go.jp/use/literacy/ssdse/#SSDSE-B)）を利用します。


In [9]:
# いくつかの列のみ抽出
df_ssdse_b <- 
  ssdse::read_ssdse_b("https://www.nstac.go.jp/sys/files/SSDSE-B-2023.csv", 
                      lang = "ja",
                      pack = FALSE) |>
  select(`SSDSE.B.2023`, `都道府県`, 
         `人口・世帯_総人口`, 
         `人口・世帯_出生数（男）`, 
         `人口・世帯_出生数（女）`,
         `教育_大学学生数`, 
         `家計_消費支出（二人以上の世帯）`,
         `家計_食料費（二人以上の世帯）`) |> 
  rename_with(~ c("year", "prefecture", 
                  "population", "birth_male", "birth_female",
                  "university_student", "spending", "food_expenses")) |>
  mutate(prefecture = zipangu::convert_prefecture(prefecture, to = "roman"))

df_ssdse_b2020 <- 
  df_ssdse_b |> 
  filter(year == 2020)

In [10]:
# データの確認
glimpse(df_ssdse_b)

### 棒グラフ


In [11]:
df_ssdse_b2020 |>
  ggplot() +
  aes(prefecture, population) +
  geom_bar(stat = "identity")

In [12]:
df_ssdse_b2020 |>
  ggplot() +
  aes(prefecture, population) +
  geom_bar(stat = "identity") +
  # 棒を縦に並べる
  coord_flip()

In [13]:
df_ssdse_b2020 |>
  # おおよその緯度の順番に配置する
  mutate(prefecture = forcats::fct_rev(forcats::fct_inorder(prefecture))) |>
  ggplot() +
  aes(prefecture, population) +
  geom_bar(stat = "identity") +
  coord_flip()

### 散布図

散布図は、2変数間の関係を表現するために用いられるグラフです。例えば、体長と体重の関係を表現することで、体長が大きい動物は体重も大きい傾向にあることがわかります。


In [14]:
p <- 
  df_ssdse_b2020 |>
  ggplot() +
  aes(food_expenses, spending) +
  geom_point()

In [15]:
# 都道府県名を表示
p +
  geom_text_repel(aes(label = prefecture))

In [16]:
# 四国4県のポイントをハイライト
p  +
  geom_text(data = df_ssdse_b2020 |> 
                    filter_shikoku(), 
                  aes(label = prefecture)) +
  gghighlight(prefecture %in% c("Tokushima-ken", "Kagawa-ken", "Ehime-ken", "Kochi-ken"))

### 折れ線グラフ


In [17]:
p <- 
  df_ssdse_b |>
  filter_shikoku() |>
  ggplot() +
  aes(year, population, group = prefecture, color = prefecture) +
  geom_line()

In [18]:
# 任意の色指定
# x軸の表記を変更
p +
  scale_color_manual(
    values = c(`Tokushima-ken` = "#953F56", 
               `Ehime-ken` = "#0F1347", 
               `Kagawa-ken` = "#77835E", 
               `Kochi-ken` = "#BC974E")) +
  scale_x_continuous(
    breaks = seq.int(2008, 2020, by = 2))

### 箱ひげ図

箱ひげ図は、データの分布（最小値、中央値を含めた四分位点、最大値）を表現するために用いられるグラフです。動物の分類群など、離散値のデータを可視化することで分類群での比較が可能となります。


In [19]:
p_core <- 
  df_ssdse_b |>
  filter_shikoku() |>
  ggplot() +
  aes(prefecture, spending)

In [20]:
p_core +
  geom_boxplot()

バイオリンプロット ... 具体的なデータの分布を確認


In [21]:
p_core + 
  geom_violin()

### ヒストグラム


In [22]:
df_ssdse_b2020 |>
  ggplot() +
  aes(x = spending) +
  geom_histogram(bins = 10)

In [23]:
df_ssdse_b |>
  filter_shikoku() |>
  filter(between(year, 2015, 2020)) |>
  ggplot() +
  aes(prefecture, population) +
  geom_bar(stat = "identity",
           aes(group = year, fill = year),
           position = "dodge") +
  scale_fill_viridis_c() +
  coord_flip()

### 座標系


In [24]:
p <-
  ggplot(data = penguins) +
  aes(x = flipper_length_mm,
      y = bill_length_mm) +
  geom_point()

p +
  coord_fixed(ratio = 1)
p +
  coord_flip()

In [25]:
p <-
  df_ssdse_b2020 |>
  filter_shikoku() |>
  ggplot() +
  aes(x = 1, population, fill = prefecture) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = c(`Tokushima-ken` = "#953F56", 
               `Ehime-ken` = "#0F1347", 
               `Kagawa-ken` = "#77835E", 
               `Kochi-ken` = "#BC974E"))

In [26]:
p

In [27]:
p +
  coord_polar(theta = "y", start = 0)

### スケール


In [28]:
df_ssdse_b |>
  filter_shikoku() |>
  ggplot() +
  aes(year, population,
      group = prefecture,
      color = prefecture) +
  geom_line() +
  scale_x_continuous(
    breaks = seq.int(2008, 2019, by = 2)) +
  scale_y_log10() +
  scale_color_viridis_d()

### その他


In [29]:
df_ssdse_b |>
  ggplot(aes(year, university_student, color = prefecture)) +
  geom_line()

In [30]:
# 人口1万人あたり
df_ssdse_b |>
  mutate(university_student_per10000 = (university_student / population) * 10000) |>
  ggplot(aes(year, university_student_per10000, color = prefecture)) +
  geom_line(show.legend = FALSE) +
  scale_y_log10() +
  gghighlight(prefecture == "Tokushima-ken")

可視化したい項目に応じてデータを整える


In [31]:
df_ssdse_b_pop <-
  df_ssdse_b |> 
  filter_shikoku() |> 
    select(year, prefecture, starts_with("birth_")) |> 
    tidyr::pivot_longer(cols = starts_with("birth_"),
                        names_to = "gender",
                        values_to = "population",
                        names_prefix = "birth_")

head(df_ssdse_b_pop, n = 3)

In [32]:
# facet
df_ssdse_b_pop |>  
  ggplot() +
  aes(year, population) +
  geom_line(aes(group = prefecture, color = prefecture)) +
  facet_wrap(~ gender,
             ncol = 2)

## データ可視化のテクニックと注意点


In [33]:
ggplot(penguins) +
  aes(x = body_mass_g, y = bill_length_mm) +
  geom_point()

ggplot(penguins) +
  aes(bill_length_mm, flipper_length_mm) +
  geom_point(aes(color = species)) +
  geom_smooth(aes(group = species), method = "lm", se = FALSE)